# Token classification (PyTorch)

Install the Datasets, Evaluate, Transformers and Accelerate libraries to run this notebook.

In [1]:
!pip install -q datasets evaluate transformers[sentencepiece]
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.3 MB/s eta 0:00:00


---

To run the training on a TPU, you will need to uncomment and run the following line:

In [2]:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

You will need to setup Git, and input your email and name below.

In [3]:
# !apt install git-lfs

In [4]:
# !git config --global user.email "sambit.mukherjee@gmail.com"
# !git config --global user.name "sadhaklal"

You will also need to be logged in to the Hugging Face Hub. Execute the following cell and enter your credentials.

In [5]:
# from huggingface_hub import notebook_login

# notebook_login()

In [6]:
# !git config --global credential.helper store

---

We shall use the CoNLL-2003 (Conference on Computational Natural Language Learning) dataset.

Dataset page: https://huggingface.co/datasets/conll2003

In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
raw_datasets

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [8]:
example0 = raw_datasets['train'][0]
for k, v in example0.items():
    print(f"{k}: {v}")

id: 0
tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
pos_tags: [22, 42, 16, 21, 35, 37, 16, 21, 7]
chunk_tags: [11, 21, 11, 12, 21, 22, 11, 12, 0]
ner_tags: [3, 0, 7, 0, 0, 0, 7, 0, 0]


In [9]:
len(example0['tokens']), len(example0['pos_tags']), len(example0['chunk_tags']), len(example0['ner_tags'])

(9, 9, 9, 9)

In [10]:
features = raw_datasets['train'].features
features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [11]:
ner_feature = features['ner_tags']
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

**Note:** `ner_feature` is a `Sequence` object with a `ClassLabel` object inside it. In other words, it's a sequence of class labels.

In [12]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [13]:
words = example0['tokens']
labels = example0['ner_tags']
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [14]:
example4 = raw_datasets['train'][4]
for k, v in example4.items():
    print(f"{k}: {v}")

id: 4
tokens: ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
pos_tags: [22, 27, 21, 35, 12, 22, 22, 27, 16, 21, 22, 22, 38, 15, 22, 24, 20, 37, 21, 15, 24, 16, 15, 22, 15, 12, 16, 21, 38, 17, 7]
chunk_tags: [11, 11, 12, 13, 11, 12, 12, 11, 12, 12, 12, 12, 21, 13, 11, 12, 21, 22, 11, 13, 11, 1, 13, 11, 17, 11, 12, 12, 21, 1, 0]
ner_tags: [5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]


In [15]:
words = example4['tokens']
labels = example4['ner_tags']
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
print(line1)
print(line2)

Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O 


In [16]:
raw_datasets = raw_datasets.remove_columns(['id', 'pos_tags', 'chunk_tags'])
raw_datasets = raw_datasets.rename_column('tokens', 'words')
raw_datasets = raw_datasets.rename_column('ner_tags', 'labels')
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['words', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['words', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['words', 'labels'],
        num_rows: 3453
    })
})

In [17]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

True

In [18]:
tokenizer.model_max_length

512

In [19]:
example0 = raw_datasets['train'][0]
for k, v in example0.items():
    print(f"{k}: {v}")

words: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
labels: [3, 0, 7, 0, 0, 0, 7, 0, 0]


In [20]:
inputs = tokenizer(example0['words'], is_split_into_words=True)
inputs

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

**Note:** An example in a Hugging Face `Dataset` (whether raw or tokenized) is a dict. But two things are `BatchEncoding` objects:

- The return value of any fast tokenizer (as we see above).
- A batch obtained from a data loader.

In [22]:
print(inputs.tokens())

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


Alt:

In [23]:
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


In [24]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Some context to understand the function below:

In [25]:
len(example0['labels']), len(inputs.word_ids())

(9, 12)

Each label in `raw_datasets` corresponds to a word, whereas each word ID corresponds to a subword.

In [26]:
# Reminder:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Notice that the `'B-XXX'` tokens have odd indices.

In [28]:
def align_labels_with_tokens(labels, word_ids):
    """
    This function returns labels corresponding to each subword (for a single example).
    Args:
        - labels: The labels corresponding to each word.
        - word_ids: The word IDs corresponding to each subword.
    """
    new_labels = []
    current_word_id = None
    for word_id in word_ids:
        # This loop goes over each word ID within an example.
        if word_id is None:
            # Special token...
            new_labels.append(-100)
        else:
            if word_id != current_word_id:
                # Start of a new word...
                label = labels[word_id]
                new_labels.append(label)
            else:
                # Continuation of the same word...
                label = labels[word_id]
                if label % 2 == 1:
                    # If the label is 'B-XXX', then change it to 'I-XXX':
                    label += 1
                new_labels.append(label)
        current_word_id = word_id
    return new_labels

**Note:** An entity in `raw_datasets` may be comprised of a `'B-XXX'` word & zero or more `'I-XXX'` words. The `if label % 2 == 1:` block above is ensuring that the labels of tokens that do not begin words are `'I-XXX'` (even if the labels of the associated words are `'B-XXX'`).

In [29]:
print(example0['words'])
print(example0['labels'])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [30]:
# Test:
print(inputs.tokens())
print(align_labels_with_tokens(example0['labels'], inputs.word_ids()))

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


Let's see what a `BatchEncoding` object's `word_ids(idx)` method call returns.

In [31]:
examples = raw_datasets['train'][:2]
for k, v in examples.items():
    print(f"{k}: {v}")

words: [['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn']]
labels: [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]


In [32]:
encoding = tokenizer(examples['words'], truncation=True, max_length=tokenizer.model_max_length, is_split_into_words=True)
encoding

{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], [101, 1943, 14428, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [33]:
encoding.word_ids(0)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [34]:
encoding.word_ids(1)

[None, 0, 1, None]

In [35]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['words'], truncation=True, max_length=tokenizer.model_max_length, is_split_into_words=True)
    all_labels = examples['labels']
    new_labels = []
    for i, labels in enumerate(all_labels):
        # This loop goes over each of the examples.
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [36]:
# Test:
tokenize_and_align_labels(examples)

{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], [101, 1943, 14428, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [37]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True, remove_columns=['words'])
tokenized_datasets

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3453
    })
})

In [38]:
example0 = tokenized_datasets['train'][0]
for k, v in example0.items():
    print(f"{k}: {v}")

labels: [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
input_ids: [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


---

In [ ]:
from transformers import DataCollatorForTokenClassification

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
[tokenized_datasets["train"][i] for i in range(2)]

[{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'input_ids': [101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100],
  'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'attention_mask': [1, 1, 1, 1],
  'input_ids': [101, 1943, 14428, 102],
  'labels': [-100, 1, 2, -100],
  'token_type_ids': [0, 0, 0, 0]}]

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
            119,   102],
         [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
              0,     0]]),
 'labels': tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
         [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
for i in range(2):
  print(tokenizer.decode(batch["input_ids"][i]))

[CLS] EU rejects German call to boycott British lamb. [SEP]
[CLS] Peter Blackburn [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [ ]:
for i in range(2):
  print(tokenized_datasets["train"]["labels"][i])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.4 MB/s eta 0:00:011
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=8af3fec966dc794db6d6f613d41684a57f4114fbc94b011ea7bf8171357880f4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'f1': 0.6666666666666666,
  'number': 2,
  'precision': 1.0,
  'recall': 0.5},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 0.8888888888888888,
 'overall_f1': 0.8,
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666}

Notice that the `predictions` and `references` arguments are both lists of lists.

In [ ]:
import numpy as np

In [ ]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  # Remove special tokens and convert to string labels:
  true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
  true_predictions = [
    [label_names[p] for p, l in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
  ]
  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": all_metrics["overall_precision"],
      "recall": all_metrics["overall_recall"],
      "f1": all_metrics["overall_f1"],
      "accuracy": all_metrics["overall_accuracy"]
  }

**Note:** This function computes the metrics at a token level (not at a word level), leaving out special tokens. However, the illustration above the function definition computes the metrics at a word level!

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
model.config.num_labels

9

In [ ]:
from transformers import TrainingArguments

In [ ]:
args = TrainingArguments(
    "bert-finetuned-ner",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=True,
    # push_to_hub=False
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

HTTPError: ignored

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14042
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5268


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.084400,0.067845,0.917781,0.935544,0.926577,0.982325
2,0.033300,0.064403,0.929801,0.945136,0.937406,0.984694
3,0.024100,0.061323,0.935239,0.952710,0.943893,0.986401


***** Running Evaluation *****
  Num examples = 3251
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-1756
Configuration saved in bert-finetuned-ner/checkpoint-1756/config.json
Model weights saved in bert-finetuned-ner/checkpoint-1756/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-1756/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-1756/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 8
Saving model checkpoint to bert-finetuned-ner/checkpoint-3512
Configuration saved in bert-finetuned-ner/checkpoint-3512/config.json
Model weights saved in bert-finetuned-ner/checkpoint-3512/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-3512/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-3512/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 8
Saving model checkpoin

TrainOutput(global_step=5268, training_loss=0.067127833062261, metrics={'train_runtime': 1327.5659, 'train_samples_per_second': 31.732, 'train_steps_per_second': 3.968, 'total_flos': 921690774203400.0, 'train_loss': 0.067127833062261, 'epoch': 3.0})

In [ ]:
# trainer.push_to_hub(commit_message="Training complete")

---

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

len(train_dataloader), len(eval_dataloader)

(1756, 407)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps

In [ ]:
from torch.optim import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

In [ ]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [ ]:
from transformers import get_scheduler

In [ ]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
# from huggingface_hub import Repository, get_full_repo_name

# model_name = "bert-finetuned-ner-accelerate"
# repo_name = get_full_repo_name(model_name)
# repo_name

In [ ]:
# output_dir = "bert-finetuned-ner-accelerate"
# repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
  predictions = predictions.detach().cpu().clone().numpy()
  labels = labels.detach().cpu().clone().numpy()

  true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
  true_predictions = [
    [label_names[p] for p, l in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
  ]
  return true_predictions, true_labels

In [ ]:
from tqdm.auto import tqdm
import torch

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
  # Training:
  model.train()
  for batch in train_dataloader:
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  # Evaluation:
  model.eval()
  for batch in eval_dataloader:
    with torch.no_grad():
      outputs = model(**batch)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]

    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)

    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    metric.add_batch(predictions=true_predictions, references=true_labels)

  results = metric.compute()
  print(f"Epoch {epoch}:", {key: results[f"overall_{key}"] for key in ["precision", "recall", "f1", "accuracy"]})

  # Saving and uploading:
  # accelerator.wait_for_everyone()
  # unwrapped_model = accelerator.unwrap_model(model)
  # unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
  # if accelerator.is_main_process:
    # tokenizer.save_pretrained(output_dir)
    # repo.push_to_hub(commit_message=f"Training in progress epoch {epoch}", blocking=False)

  0%|          | 0/5268 [00:00<?, ?it/s]

Epoch 0: {'precision': 0.9075314491096226, 'recall': 0.9348704140020195, 'f1': 0.9209980933432811, 'accuracy': 0.9837228468829105}
Epoch 1: {'precision': 0.9210612512283001, 'recall': 0.9464826657691013, 'f1': 0.9335989375830013, 'accuracy': 0.9849885206334256}
Epoch 2: {'precision': 0.9248725957586718, 'recall': 0.9468192527768429, 'f1': 0.9357172557172558, 'accuracy': 0.9853858833225407}


**Note:** We're not using the `compute_metrics` function that we defined.

---

In [ ]:
from transformers import pipeline

In [ ]:
# Replace this with your own checkpoint:
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp32umog7y


Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

storing https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/89b5cbe52d0bb00707c92604aa32923af1e03e431c6dc7755afa8e12736b8611.61f3554ca83a9cd2ab0687ff37a7d0f4065bc064c4f297e5d2d42e322e4c5f26
creating metadata file for /root/.cache/huggingface/transformers/89b5cbe52d0bb00707c92604aa32923af1e03e431c6dc7755afa8e12736b8611.61f3554ca83a9cd2ab0687ff37a7d0f4065bc064c4f297e5d2d42e322e4c5f26
loading configuration file https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/89b5cbe52d0bb00707c92604aa32923af1e03e431c6dc7755afa8e12736b8611.61f3554ca83a9cd2ab0687ff37a7d0f4065bc064c4f297e5d2d42e322e4c5f26
Model config BertConfig {
  "_name_or_path": "huggingface-course/bert-finetuned-ner",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpoint

Downloading:   0%|          | 0.00/411M [00:00<?, ?B/s]

storing https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/53be13866b5e6a7bf270c198a504ba00a1d6c99765a5425c8d8a6f67474c59db.34b5567d0f0878a5afa5157f6e9b2fe7742287d066ae18b16a646cba224cb46f
creating metadata file for /root/.cache/huggingface/transformers/53be13866b5e6a7bf270c198a504ba00a1d6c99765a5425c8d8a6f67474c59db.34b5567d0f0878a5afa5157f6e9b2fe7742287d066ae18b16a646cba224cb46f
loading weights file https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/53be13866b5e6a7bf270c198a504ba00a1d6c99765a5425c8d8a6f67474c59db.34b5567d0f0878a5afa5157f6e9b2fe7742287d066ae18b16a646cba224cb46f
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at huggingface-course/bert-finetuned-ner.
If your task 

...

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

storing https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/450ab56275366591009a03ebe21bfa2523a89a590ac2e4b569920025b849ecf5.1f9d100b22551a7009fb51f1fadb9158af5db04f4c188aceecaa745a1917c983
creating metadata file for /root/.cache/huggingface/transformers/450ab56275366591009a03ebe21bfa2523a89a590ac2e4b569920025b849ecf5.1f9d100b22551a7009fb51f1fadb9158af5db04f4c188aceecaa745a1917c983
https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp4hponjd1


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/1f7a04f6385a04a9c60686046244d8daaa06489d154d6523ca28c5d8430c74c0.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/1f7a04f6385a04a9c60686046244d8daaa06489d154d6523ca28c5d8430c74c0.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/dbadb9cd77a95bc4e921c9457f6c9f87f9654e89c139503e43f3c6abd4aef018.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/huggingface-course/bert-finetuned-ner/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/450ab56275366591009a03ebe21bfa2523a89a590ac2e4b569920025b849ecf5.1f9d100b22551a7009fb5

[{'end': 18,
  'entity_group': 'PER',
  'score': 0.9988506,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.9647625,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.9986118,
  'start': 49,
  'word': 'Brooklyn'}]